In [1]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [2]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [3]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 10
n_iter = 300000 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [10]:
# import impl.RNN as nn

# import impl.LSTM as nn

# import impl.GRU as nn

import impl.GRU2 as nn

net = nn.GRU2(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [11]:
solver.adam_rnn(
    net, X, y,
    alpha=alpha,
    mb_size=time_step,
    n_iter=n_iter,
    print_after=print_after
)

Iter-1000 loss: 3.3874
 noueap r. Japnd Nale ir t2gund d ,ar 4epe pteonoh 7antierd ik and aa-eAarOy 3lan oconn che ns rond 


Iter-2000 loss: 2.7317
om the in le, Japal io th ohith- "rgise tapceplalymy thd -nte undt ins I-maalin sopsc bolloTf Japen 


Iter-3000 loss: 2.2992
rof Japan'p lioc nskourd-the Eeal. Japangicinn, JapanysceRendes nd HResiont eculobr i. che the nd St


Iter-4000 loss: 2.0041
urtoy oxed cernomgest enerpensin pand wop1d:ind as irgonClagnti, onod Iturs es girgd HArut, oskghbvt


Iter-5000 loss: 1.7843
ction on Japyn sivesP ca esta Nevin of 19tO in Sna (fef m: As and arito falWea 194l whoch penturys r


Iter-6000 loss: 1.6114
ourty Dmat ry of teres, whes a "omper ur the Sed Japan, if tounont. furlinost. Ocen isgtht les reane


Iter-7000 loss: 1.4472
and Seaka af i95banand 2本0lan"8f1TTh Naonkey Tonf ciano plalidan kutiy O7 ulebot enss lithe U2isa, S


Iter-8000 loss: 1.3227
in the ntov lfof litn is and en en in hine fide ty ons ripity, whest Asiof, Jopan has mateong t

Iter-25000 loss: 0.5763
rd Asina and ulesh coxth first. Japane World Warly war II lob2 thinasinglinn perchis ,astre Aria dov


Iter-26000 loss: 0.5495
 audeanTs of intirna it as, E8ptoped Japanese War on Tie alithy sured. Thits in the ciped-Japan's hi


Iter-27000 loss: 0.5305
histor. Jatanusesulero-labanes. The natwon the Empter of OJalandes iolarchin lagger of Japan'o and t


Iter-28000 loss: 0.5141
vexcand in 1853 whe G8, in Eassed Japan the pomy the kokitan and its rightst ciOly-d Hokkok onflitio


Iter-29000 loss: 0.4994
itictuaided atof uraling romes farst tomina. Japan was rtyoptang the world's mercand intr ateJapan t


Iter-30000 loss: 0.4797
f Japanese War of Okhe fourth largest met ronsed the Restu teo e perted Sinte and It. The caine inte


Iter-31000 loss: 0.4748
ate thitha, Japan is and the GE. Thilatio tever the Empiro fimbowar Dexpaler Asitasikaicalald inteca


Iter-32000 loss: 0.4733
kkaidy, chesEagas fountred, in the oromear. Inku2sic archiky, Suncen ard Nipy of of low

Iter-49000 loss: 0.3584
 ited Tokyo, the country it Japanas inareld ItpeIcanas Gafhibgland fourth-largest economy by pecceld


Iter-50000 loss: 0.3299
shu opnJapanc opely and fourth-largest economins ongCrendivican sortex and andex, cassld rtetoluty. 


Iter-51000 loss: 0.3174
ectero Easty reUn 12o sme mogrent rendernast. Japaritioulepiogar ho pare of bokxtiti. Japan is an Ch


Iter-52000 loss: 0.3126
ty 19trly as a dintionter of 1937 fre G8, and ear. Japancaba wil in the vorth largest omptre flom or


Iter-53000 loss: 0.3011
anese: 日本 Nipory, the periec or andexcan is the hipxth s of inoryd roverty th laggembititan and Nove


Iter-54000 loss: 0.2939
my by t of JapanNitrias, rectore ,at Ind1Ja, wrictand coundud ing porer the G8, ande Olympiritaric b


Iter-55000 loss: 0.2890
wo legiter In thiceches uriteres, wishy of 6,852 isla and ioplof in alel pirotity pineto-sings of Ht


Iter-56000 loss: 0.2739
e forpth largest mente chilitety, wariny. st the Global wopitaly of the Empire ofgestri

Iter-73000 loss: 0.2276
anji that makate of Japan is country Brancl Impirial country in the sumbolitares a the World War I a


Iter-74000 loss: 0.2098
s. Japan is a member of the UN, the G7, the G8, andlStar, is the Firper Paicons inurevinos onfictoru


Iter-75000 loss: 0.2181
to expand is code, has chatoly war, is the fitbof century CE. Influence from other the world's limal


Iter-76000 loss: 0.2231
 The first cound itice II inorth largest militanatiou, wowrodimeat revenot onte palion. Japan is a n


Iter-77000 loss: 0.2029
ed in 1947, the G8, ard ding of the pame of Worlly Brand dt increded forss a membar CEunared dilitir


Iter-78000 loss: 0.2221
an Developmo, iasurares, wrictaing of the Global 120this shurHchisecald in the Pales. The fourth-lar


Iter-79000 loss: 0.2125
ond Sino-Japanes inuro-deveno. Japan ear of in the Pacis the world's thi deled Rts reatery bal Chine


Iter-80000 loss: 0.2113
he 12th centuriest vild itiond. The Upper Paled bountry in 19th arch pares in the world

Iter-97000 loss: 0.2001
ke up 98.5% of Japan's name thic an gron the world's fourth-largest exporter and fourth-largest expo


Iter-98000 loss: 0.1629
he world's fourth-largest expornly reatisn asorings insthil hoth the world's eight wro tee Russhas d


Iter-99000 loss: 0.1921
 aEmb. Japan is rodsita laggeater devex an hiercyurided Wor Ixperile. The coun, war regtote sing mil


Iter-100000 loss: 0.1991
the duvital city of Japan is in Chinese history texts from the 1st cenoude, in the eard of 6,852 Jap


Iter-101000 loss: 0.1703
ple largest arex the nami. Japan ean, the world's eisthe alowith the Sucmens and the world'sil ith t


Iter-102000 loss: 0.1726
h the Summer and Win6, and rankainal city of Japan's history 9l1s a divi.e, an the late of the Seato


Iter-103000 loss: 0.1964
r paries in econstialleder as a dungionte silitary st Hurchas itiored the f The porte tion of the na


Iter-104000 loss: 0.1795
lee the first ceasired in the forld and the world's largest metrexparion. Japan is

...

Iter-193000 loss: 0.1102
evisitare ceveral to encesaitaly which includes Tokyo and several surroundins wired into a Oly s off


Iter-194000 loss: 0.1231
the 12th century unting pome a deend Apportery. Altoto Emperor and a gento oviges rollowed largest. 


Iter-195000 loss: 0.1349
res in the nare of u2016 nancedital country in East Asia. Located in the Pacific Ocean, which came t


Iter-196000 loss: 0.1336
ed sixth in the Global Competitivenes of internal conflict and insurrection forlese War of 1937 expa


Iter-197000 loss: 0.1085
onal monarchy with and Human Development Index who regions the 12t bolderit. Japan to enced in the w


Iter-198000 loss: 0.1338
tary Im Emperor and Taiwan in the south. The pagy, pame forrclatude military is the world's largest 


Iter-199000 loss: 0.1312
th largest. Japanest ligrofhis toming Sines of by pured Japan has fomeracesef of rence folle pured i


Iter-200000 loss: 0.1210
hess rigsh Japan ts 17ths warhest th the world's largest military budget, used 

Iter-217000 loss: 0.1344
r and fourth-largest economy by nominal GDP and the world's fourth-largest economy by purchasing mil


Iter-218000 loss: 0.1112
mpic paro of Japan's toming purthital cimat ond Nihon-kkancy, wil9.1ban (Japanese War of 1937 expand


Iter-219000 loss: 0.1064
e wored's the world's tentio . The Seac -of in 1868 and which includes Tokyo and several surroundila


Iter-220000 loss: 0.1216
pan to end it is often was the Emperor as a divine symbol of the nation. In the late 19th and early 


Iter-221000 loss: 0.1269
 its of int th the Numas of by the Rusish oved in the world. The Greateries a lowes of intend io the


Iter-222000 loss: 0.1144
the nation. In the late Eastl t"ky of China, followed by periods of isolation in the Meipered inshic


Iter-223000 loss: 0.0993
 the world's third-largest economy by purchasing power paried ofhinnd isll whirR sevela. The country


Iter-224000 loss: 0.1208
Sea of Okhots in the Global Competity in the core city of Tokyo, the core city 

Iter-241000 loss: 0.1315
ed Asian country uD and HumankDeied of internal Diled the highesthit lo War of 193 frrochelder Nihon


Iter-242000 loss: 0.1030
gs of Hiros-keo founty, which grom in 1853 when a United in the Pacific Ocean, it lies to the east o


Iter-243000 loss: 0.1270
n Europe, has characterized Japan's hast country in East Asia. Locate of 126 million people live in 


Iter-244000 loss: 0.1134
land area. Asia. Located in the Pacif the East China Sea and Taiwan in the sore city of Tokyo, the c


Iter-245000 loss: 0.1150
n is ranked first in the fourth leading global city in the world's fourth-largest expanded into part


Iter-246000 loss: 0.1212
in 1947, Japan has maintained a unitarised for pelfereal pepec aky. The kanji that make up Japan's n


Iter-247000 loss: 0.1063
il 1868, Japan was the first country in the world, and ranked stvenfloment wre tived the country wit


Iter-248000 loss: 0.1078
. che world's fourth-largest economy by pucttot. Snaceatitational minal GDP and

Iter-265000 loss: 0.1000
 "State of the Russo-Japanese War IIII toldeved ism. Wor Asiates thar dexpart of World War II in mai


Iter-266000 loss: 0.1007
ino, Kokled ism. it are port and the fitht country in the Global Peace Index. Japan was rleching fro


Iter-267000 loss: 0.1368
or was restongh Tokyo andisunamwor Arich saitary by from the 1st centary cinso toxt urrenting the a,


Iter-268000 loss: 0.1146
s Tokyo and several surrounding prefectures in eight regions. The population of 126 million is the N


Iter-269000 loss: 0.1121
acific Ocean owed peatimin the world, and ranked first in the Cound Nihons gry, pulled in the early 


Iter-270000 loss: 0.1093
 urban agglomeration elopingsmicto with the Sea, Japan extate Dieolal sexparked. Japan is a stratovo


Iter-271000 loss: 0.1120
roclaimed, world War I allicallG20. The four larges the Sea of Okhotsk in the Countin"ind Sincerad i


Iter-272000 loss: 0.1255
tropolitan areates military budg oretal country in the Global Peace Index. Japa

Iter-289000 loss: 0.1226
islands. The four larital conflicly of Japan's Mex. The kanji 1ith the Sea of Okhotsk in the north c


Iter-290000 loss: 0.1096
ird lowest in the world. The Greater Tokyo Area, which wangest in the number. Japan was proclaimed, 


Iter-291000 loss: 0.1123
came to an end in 1945 followit. Japan was ruled by successive feudal military shoguns who ruled in 


Iter-292000 loss: 0.1141
 by period of isolation in the early 17th century, which was endpen, ions, mainly Imperial China, fo


Iter-293000 loss: 0.1236
ich maino. The East Areater Tokyo Area, which in the Global Peace Index, ran Asia. Localy of the nat


Iter-294000 loss: 0.1091
es. Japan was the first country with a high standard of living and Human Development of Japan is in 


Iter-295000 loss: 0.1086
Nagasate 19th and early 20th centuries, victories in the First Sino-Japanese War and World's laterie


Iter-296000 loss: 0.1261
terized Japan's history. From the 12th century until 1868, Japan was the first 

In [ ]:
=========================================================================
Iter-299000 loss: 0.1077
=========================================================================
are dover as the world's largest mentu CrCo 41nter anha. The Nanaly which make up an in in tont urom
=========================================================================


=========================================================================
Iter-300000 loss: 0.1066
=========================================================================
ugleat of of the 1st uctate in 1868 and the world's eighth largest military budget, asef. The first 
=========================================================================


Out[11]:
<impl.GRU2.GRU2 at 0x7f4769a8e518>

In [ ]:
=========================================================================
Iter-299000 loss: 0.1203
=========================================================================
ake up Japan's frumititan Devenolich in 1941, which came to an end in 1947, Japan's name mean "sun d
=========================================================================


=========================================================================
Iter-300000 loss: 0.1005
=========================================================================
uled by successive feudal military shoguns who ruled in the name of the Empire of Japan, which is th
=========================================================================


Out[9]:
<impl.GRU.GRU at 0x7f47741642e8>

In [ ]:
=========================================================================
Iter-299000 loss: 0.0664
=========================================================================
aintained a unitary constitutional monarchy with an Emperor and an electe me 11rlly ursha regions of
=========================================================================


=========================================================================
Iter-300000 loss: 0.0640
=========================================================================
uled by sucgun". Japan is a umane CoromoWnr has IIhenthdentican perorld. The Sino-Japanese War, the 
=========================================================================


Out[7]:
<impl.LSTM.LSTM at 0x7f477416c4a8>

In [ ]:
=========================================================================
Iter-299000 loss: 0.2072
=========================================================================
aintained ingChigin", was sulangifolline puopichic Gamoko. Japan has maintained ing1gion. Aroper Pac
=========================================================================


=========================================================================
Iter-300000 loss: 0.1885
=========================================================================
ul ded Index.oNagecty uds oftel militarism in Eurby, war Astand end Nipacess ud the "Land itintevolc
=========================================================================


Out[5]:
<impl.RNN.RNN at 0x7f4774164358>